[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pachyderm/examples/blob/master/object-detection/Pachyderm%20with%20WandB%20Object%20Detection.ipynb)
<!--- @wandbcode{pach_wandb} -->

# Object Detection: Pachyderm with Weights and Biases

This example illustrates a common Object Detection use case using [Pachyderm](https://www.pachyderm.com/), [Weights and Biases](https://docs.wandb.ai/), and [Lightning Flash](https://lightning-flash.readthedocs.io/en/latest/). 

<p align="center">
	<img src='images/wandb_diagram.png' width='500' title='Pachyderm'>
</p>

This demo mimics the object detection [example from Lightning Flash](https://lightning-flash.readthedocs.io/en/stable/reference/object_detection.html#example). We extend the example to use to log our training runs with Weights and Biases. 

<p align="center">
	<img src='images/training_diagram.png' width='500' title='WandB'>
</p>

## Step 0: Setup
Before getting started, make sure you have a Pachyderm cluster and a W&B account. 

1. Create a [Pachyderm cluster](https://docs.pachyderm.com/latest/getting_started/).
2. Create a [W&B Account](https://wandb.ai/) 

## Step 1: Upload Dataset

In [ ]:
!wget https://github.com/zhiqwang/yolov5-rt-stack/releases/download/v0.3.0/coco128.zip
!unzip coco128.zip

In [3]:
!pachctl create repo coco128

In [ ]:
!pachctl put file -r coco128@master:/coco128/ -f coco128

## Step 2: Train Model
Our model pipeline will train an object detection model on our dataset. 

Our model pipeline will train an object detection model on our dataset, tracking it with [Weights and Biases](https://wandb.ai/site) by configuring a Pachyderm secret and deploying the [model_wandb.json pipeline](pachyderm/model_wandb.json).

You can see more about the W&B integration [here](https://github.com/pachyderm/examples/tree/master/weights_and_biases).

In [ ]:
# Create wandb secret
!pachctl create secret -f wandb_secret.json

In [5]:
!pachctl create pipeline -f pachyderm/model_wandb.json

## Step 3: Update Training Dataset
Now that we've deployed our training pipeline, any changes to our input dataset will trigger a new training job. All training runs will log to Weights and Biases, with the job name referencing the Run ID (Global ID) in Pachyderm. 

In [ ]:
!pachctl put file -r coco128@master:/coco128/images/train2017/dog1.jpeg -f images/dog1.jpeg

Now, everytime we sync our manual edits from Label Studio, it will automatically update our source dataset and kick off a retraining of our object detection model. 

## Clean up

In [2]:
!pachctl delete pipeline model
!pachctl delete repo coco128